<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/without-finetuning/askApatient_3_pretraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [7]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (60) SSL certificate problem: certificate has expired
More details here: https://curl.haxx.se/docs/sslcerts.html

curl failed to verify the legitimacy of the server and therefore could not
establish a secure connection to it. To learn more about this situation and
how to fix it, please visit the web page mentioned above.


In [8]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-3.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [13]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-3.validation.csv",header=None)
valid.head()

,0,1
0,2,left heel pain
1,2,right heel became painful
2,8,Arthritis
3,8,arthritis
4,8,arthritis


In [14]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-3.test.csv",header=None)
test.head()

,0,1
0,2,heel pain
1,5,rapid heart beat
2,6,diastolic dysfunction
3,8,arthritis
4,8,arthritis


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine-tuning_language_model_asapatient_twadr_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [0]:
data_lm.save()
data_clas.save()

In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
# learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [0]:
# learn.fit_one_cycle(1, 1e-2)

In [0]:
# learn.fit_one_cycle(1, 1e-2)

In [0]:
# learn.fit_one_cycle(1, 1e-3)

In [0]:
# learn.fit_one_cycle(1, 1e-3)

In [0]:
# learn.save_encoder('bigtweet_fold0_first')

In [28]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
# learn.load_encoder('bigtweet_fold0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.543191,5.984186,0.151399,01:04


In [29]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.776691,17.840250,0.010178,01:17


In [30]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.867679,9.630992,0.118321,03:20


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.243853,5.194627,0.301527,03:22
1,2.807707,5.478397,0.311705,03:19


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.371894,6.618391,0.326972,03:13
1,1.995735,5.804506,0.296438,03:17


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.972954,7.231383,0.278626,03:12
1,1.704843,4.996917,0.366412,03:28


In [34]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.630334,7.871685,0.240458,03:32
1,1.400826,5.318252,0.370229,03:23


In [35]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.293469,3.587460,0.498728,03:30
1,1.189739,3.218047,0.501272,03:19


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.055748,3.080094,0.552163,03:28
1,1.052790,3.549911,0.480916,03:34
2,0.889939,3.108615,0.559796,03:22
3,0.786286,2.947937,0.589059,03:12


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.812967,2.466512,0.625954,03:20
1,0.764846,3.310977,0.543257,03:15
2,0.703978,2.538643,0.618321,03:13
3,0.579869,2.469496,0.622137,03:19


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.592774,3.284367,0.571247,03:10
1,0.651600,2.635574,0.610687,03:21
2,0.558600,2.178156,0.653944,03:24
3,0.471572,2.150424,0.659033,03:14


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.466587,2.244142,0.646310,03:28
1,0.504787,2.328973,0.659033,03:09
2,0.483940,1.847833,0.729008,03:22
3,0.367081,1.851561,0.726463,03:24


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.383889,1.898250,0.697201,03:15
1,0.407764,2.174860,0.679389,03:07
2,0.391030,1.899012,0.723919,03:18
3,0.326983,1.820542,0.726463,03:41


In [41]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.350744,1.824335,0.730280,03:41
1,0.418613,1.737610,0.744275,03:45
2,0.342648,1.789154,0.750636,03:37
3,0.278011,1.700406,0.750636,03:39


In [42]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.311907,1.672892,0.760814,03:45
1,0.359378,1.744540,0.746819,03:37
2,0.302125,1.765126,0.751908,03:29
3,0.280727,1.704923,0.758270,03:54


In [0]:
# save the best model

learn.save_encoder('ask-0-tuning-bigtweet')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [45]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[2, 324, 6, 8, 8, 8, 49, 396, 330, 27, 27, 159, 317, 289, 31, 34, 877, 100, 289, 289, 289, 289, 289, 289, 289, 37, 37, 49, 325, 49, 683, 49, 710, 49, 49, 49, 49, 49, 49, 49, 53, 53, 53, 60, 61, 61, 491, 62, 62, 77, 289, 71, 72, 27, 76, 209, 265, 289, 79, 289, 289, 289, 79, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 289, 289, 289, 289, 80, 80, 80, 89, 531, 491, 91, 289, 94, 94, 94, 289, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 902, 96, 98, 98, 289, 100, 100, 100, 100, 861, 104, 337, 877, 108, 289, 108, 289, 289, 289, 189, 117, 415, 898, 120, 122, 355, 125, 125, 125, 125, 125, 125, 125, 289, 289, 289, 343, 343, 131, 139, 139, 139, 140, 226, 142, 151, 151, 429, 289, 265, 163, 163, 167, 419, 337, 289, 289, 172, 289, 172, 172, 471, 173, 652, 182, 182, 182, 182, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 187, 187, 187, 187, 187, 187, 187, 190, 190, 190, 190, 190, 192, 193, 193, 23, 197, 197, 34, 1

In [46]:
correct_num = 0

for i in range(0, 866):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

593
0.684757505773672
